In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('car data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
#checking missing or null values
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
final_data = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_data.head()

In [ ]:
final_data['Current_Year']=2021
final_data.head()

In [ ]:
final_data['No_of_Years'] = final_data['Current_Year']-final_data['Year']

In [ ]:
final_data.head()

In [ ]:
final_data.columns

In [ ]:
final_data.drop(['Year'],axis=1,inplace=True)

In [ ]:
final_data.drop(['Current_Year'],axis=1,inplace=True)

In [ ]:
final_data.head()

In [ ]:
final_data = pd.get_dummies(final_data,drop_first=True)

In [ ]:
final_data.head()

In [ ]:
final_data.corr()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(final_data)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat = final_data.corr()
top_feature = corrmat.index
plt.figure(figsize=(20,20))
g=sns.heatmap(final_data[top_feature].corr(),annot=True,cmap="RdYlGn")

In [ ]:
x = final_data.iloc[:,1:]
y = final_data.iloc[:,0]

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
#3feature importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph
feat_importances=pd.Series(model.feature_importances_,index=x.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
x_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random=RandomForestRegressor()

In [ ]:
import numpy as np

In [ ]:
#hyperparameters and n_estimator is no of trees
n_estimators = [int(x) for x in np.linspace(start = 100,stop= 1200,num = 12)]
print(n_estimators)

In [ ]:
#no of features to be considered at everyy split
max_features = ['auto','sqrt']
#max no of levels in tree
max_depth = [int(x) for x in np.linspace(5.,30,num = 6)]
#min sample req to split a node
min_samples_split = [2,5,10,15,100]
#min sample req at each leaf
min_samples_leaf = [1,2,5,10]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#create random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

In [ ]:
#use random grid to search for best hyperparameter
#firt create base tune model
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state = 42,n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)

In [ ]:
prediction = rf_random.predict(x_test)

In [ ]:
sns.distplot(y_test-prediction)

In [ ]:
plt.scatter(y_test,prediction)

In [ ]:
import pickle
file = open('car_price_model.pkl','wb')
pickle.dump(rf_random,file)